# JIF data preparation

This notebook associates JCR JIF scores to publications.

Preperations:
- Create a Python virtual environment as descibed in the repository README Setup instructions.
- Select the Python kernel from the virtual environment (venv/bin/python)
- The pub_extracts sub-folder should contain the publications extract file, e.g. "[type]_pd_extract_22.xlsx"
- The JIF_scores sub-folder should contain the latest JIF scores file, e.g. "JCR_JournalResults_12_2022_byISSN_221208.xlsx"
- Update the JIF scores file as needed

In [ ]:
# Before executing, select the python kernel from the local virtual env
# Requires the ipykernel is installed

print("Verify can run jupyter cell")

In [ ]:
# Settings

DATA_TYPE_PREFIX = "fellows"            # The name of the type of data for this run. Used to prefix the output file names.
FILENAME_PUBS = "fellows_pd_extract_22.xlsx"  # The publications file name.
FILENAME_JIF_SCORES = "JCR_JournalResults_2022_byISSN_run20221214.xlsx"   # The JIF scors file name.
DATA_PATH = "../data/data2022"        # The relative path to the data files.
YEARS_RANGE = [2017, 2022]            # The range of years to consider for processing. 



In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
from datetime import datetime
import sqlite3
print("Finished imports")

### Load the input data

In [ ]:
# Initialise the database

db_name = "jif.db"

if os.path.isfile(db_name):
    os.remove(db_name)

con = sqlite3.connect(db_name)


# Load input data

# The publications extract

filename_path = os.path.join(DATA_PATH, "pub_extracts", FILENAME_PUBS)
print(filename_path)
print(f"File exists: {os.path.isfile(filename_path)}")

wb = pd.read_excel(filename_path, sheet_name="Publications", keep_default_na=True, converters={'Year':int})
wb.to_sql("pub", con, index=False)


# JIF scores

filename_path = os.path.join(DATA_PATH, "JIF_scores", FILENAME_JIF_SCORES)
print(filename_path)
print(f"File exists: {os.path.isfile(filename_path)}")

wb = pd.read_excel(filename_path, sheet_name="JCR", keep_default_na=True)
wb.to_sql("jcr_score", con, index=False)


# Filter by year range

cur = con.cursor()

sql = "DELETE FROM pub WHERE [Year] NOT BETWEEN {0} AND {1};".format(YEARS_RANGE[0], YEARS_RANGE[1])
cur.execute(sql)

# Check contents

res = cur.execute("SELECT COUNT(*) FROM pub;")
print(res.fetchone())

res = cur.execute("SELECT COUNT(*) FROM jcr_score;")
print(res.fetchone())


# Prepare the pub table for matching to JIF scores

res = cur.execute("ALTER TABLE pub ADD [match_method] varchar(10);")
res = cur.execute("ALTER TABLE pub ADD [match_val] varchar(20);")
res = cur.execute("ALTER TABLE pub ADD [JIF_no_self_cites] decimal(10,3);")


con.commit()
con.close()


### Match the publications to JIF scores

In [ ]:
# Run matching algorithms

con = sqlite3.connect(db_name)
cur = con.cursor()


# Reset the matches

sql = "UPDATE pub SET [match_method] = NULL, [match_val] = NULL, [JIF_no_self_cites] = NULL;"
cur.execute(sql)


# Match ISSN = ISSN
sql = "UPDATE pub SET match_val = (SELECT jcr_score.ISSN FROM jcr_score WHERE jcr_score.ISSN=pub.ISSN) WHERE pub.match_val IS NULL;"
cur.execute(sql)
sql = "UPDATE pub SET match_method = 'ISSN:ISSN' WHERE match_method IS NULL AND match_val IS NOT NULL;"
cur.execute(sql)
sql = "UPDATE pub SET JIF_no_self_cites = (SELECT jcr_score.[JIF Without Self Cites] FROM jcr_score WHERE jcr_score.ISSN=pub.ISSN) WHERE pub.JIF_no_self_cites IS NULL;"
cur.execute(sql)

# Match ISSN = eISSN
sql = "UPDATE pub SET match_val = (SELECT jcr_score.eISSN FROM jcr_score WHERE jcr_score.eISSN=pub.ISSN) WHERE pub.match_val IS NULL;"
cur.execute(sql)
sql = "UPDATE pub SET match_method = 'ISSN:eISSN' WHERE match_method IS NULL AND match_val IS NOT NULL;"
cur.execute(sql)
sql = "UPDATE pub SET JIF_no_self_cites = (SELECT jcr_score.[JIF Without Self Cites] FROM jcr_score WHERE jcr_score.eISSN=pub.ISSN) WHERE pub.JIF_no_self_cites IS NULL;"
cur.execute(sql)

# Match ISSN-L = ISSN
sql = "UPDATE pub SET match_val = (SELECT jcr_score.ISSN FROM jcr_score WHERE jcr_score.ISSN=pub.[ISSN-L]) WHERE pub.match_val IS NULL;"
cur.execute(sql)
sql = "UPDATE pub SET match_method = 'ISSN-L:ISSN' WHERE match_method IS NULL AND match_val IS NOT NULL;"
cur.execute(sql)
sql = "UPDATE pub SET JIF_no_self_cites = (SELECT jcr_score.[JIF Without Self Cites] FROM jcr_score WHERE jcr_score.ISSN=pub.[ISSN-L]) WHERE pub.JIF_no_self_cites IS NULL;"
cur.execute(sql)

# Match ISSN-L = eISSN
sql = "UPDATE pub SET match_val = (SELECT jcr_score.eISSN FROM jcr_score WHERE jcr_score.eISSN=pub.[ISSN-L]) WHERE pub.match_val IS NULL;"
cur.execute(sql)
sql = "UPDATE pub SET match_method = 'ISSN-L:eISSN' WHERE match_method IS NULL AND match_val IS NOT NULL;"
cur.execute(sql)
sql = "UPDATE pub SET JIF_no_self_cites = (SELECT jcr_score.[JIF Without Self Cites] FROM jcr_score WHERE jcr_score.eISSN=pub.[ISSN-L]) WHERE pub.JIF_no_self_cites IS NULL;"
cur.execute(sql)

# Match Journal = JCR Abbreviation
sql = "UPDATE pub SET match_val = ( SELECT jcr_score.[JCR Abbreviation] FROM jcr_score \
     WHERE LOWER(jcr_score.[JCR Abbreviation])=LOWER(pub.[Journal]) ) WHERE pub.match_val IS NULL;"
cur.execute(sql)
sql = "UPDATE pub SET match_method = 'Journal:Abbrev' WHERE match_method IS NULL AND match_val IS NOT NULL;"
cur.execute(sql)
sql = "UPDATE pub SET JIF_no_self_cites = (SELECT jcr_score.[JIF Without Self Cites] FROM jcr_score WHERE LOWER(jcr_score.[JCR Abbreviation])=LOWER(pub.[Journal])) WHERE pub.JIF_no_self_cites IS NULL;"
cur.execute(sql)

# Match Journal = Journal name
sql = "UPDATE pub SET match_val = ( SELECT jcr_score.[Journal name] FROM jcr_score \
     WHERE LOWER(jcr_score.[Journal name])=LOWER(pub.[Journal]) ) WHERE pub.match_val IS NULL;"
cur.execute(sql)
sql = "UPDATE pub SET match_method = 'Journal:Journal' WHERE match_method IS NULL AND match_val IS NOT NULL;"
cur.execute(sql)
sql = "UPDATE pub SET JIF_no_self_cites = (SELECT jcr_score.[JIF Without Self Cites] FROM jcr_score WHERE LOWER(jcr_score.[Journal name])=LOWER(pub.[Journal])) WHERE pub.JIF_no_self_cites IS NULL;"
cur.execute(sql)


con.commit()
con.close()


### Save the result to files

In [ ]:
# Note: this will not save over existing file.

con = sqlite3.connect(db_name)


# Save all matches

df_matches = pd.read_sql_query("SELECT Title,Year,Labels,Journal,ISSN,[ISSN-L],JIF_no_self_cites,match_method,match_val \
    FROM pub WHERE [match_method] IS NOT NULL;", con)

filename = f"JIF_{DATA_TYPE_PREFIX}_matches.xlsx"
filename_path = os.path.join(DATA_PATH, filename)

if os.path.isfile(filename_path):
    print(f"Not saving the file. The file already exists: {filename_path}")
    print("Delete or move the old file to re-create.")
else:
    df_matches.to_excel( filename_path, index=False, header=True )
    print(f"Saved file: {filename_path}")


# Save non-matched rows (rows missing the JIF score) to excel to continue working manually.

df_missing = pd.read_sql_query("SELECT Title,Year,Labels,Journal,ISSN,[ISSN-L],JIF_no_self_cites FROM pub WHERE [match_method] IS NULL;", con)

# Add new columns for manual work
df_missing.insert(len(df_missing.columns), "Action", "")
df_missing.insert(len(df_missing.columns), "Status", "")
df_missing.insert(len(df_missing.columns), "Comment", "")
df_missing.insert(len(df_missing.columns), "Link", "")
df_missing.insert(len(df_missing.columns), "Review", "")

filename = f"{DATA_TYPE_PREFIX}_missing_JIF_check_manually.xlsx"
filename_path = os.path.join(DATA_PATH, filename)

if os.path.isfile(filename_path):
    print(f"Not saving the file. The file already exists: {filename_path}")
    print("Delete or move the old file to re-create.")
else:
    df_missing.to_excel( filename_path, index=False, header=True )
    print(f"Saved file: {filename_path}")


con.commit()
con.close()


### Categorise the JIF scores into groups

Run this after the input files have been finalised as desired.

In [ ]:
# First prepare the dataset to match the below code, by renaming the df and columns

# skipped match on fullnames

JIF_merge_fullnames = df_matches  # so remaining code remains same

JIF_merge_fullnames.rename(
    columns={
        "ISSN_x": "ISSN",
        "Title": "Full Journal Title",
        #"JCR Abbreviated Title_x": "JCR Abbreviated Title",
        "JIF_no_self_cites": "JIF",
    },
    inplace=True,
)


# Segment up the JIFs to groups

JIF_merge_fullnames["JIF"] = JIF_merge_fullnames["JIF"].fillna(-1)

JIF_merge_fullnames["JIF"] = pd.to_numeric(JIF_merge_fullnames["JIF"])
JIF_merge_fullnames["JIFcat"] = pd.cut(
    JIF_merge_fullnames["JIF"],
    bins=[-1, 0, 6, 9, 25, 1000],
    include_lowest=True,
    labels=["JIF unknown", "JIF <6", "JIF 6-9", "JIF 9-25", "JIF >25"],
)

# Need to do a group by and check the sums work

JIF_sub = JIF_merge_fullnames[["Year", "Labels", "JIFcat"]]

JIF_sub_group_inf = JIF_sub.groupby(["Year", "JIFcat"]).size().reset_index()

JIF_sub_group_inf.columns = ["Year", "JIFcat", "Count"]


# Use this to check that the sums are as expected given the original publication files
today = datetime.today().date()
year = today.year
output_path = os.path.join(DATA_PATH, f"{DATA_TYPE_PREFIX}_JIF_groups_{year}.xlsx")
JIF_sub_group_inf.to_excel(output_path)



### Create plots

In [ ]:
# Generate plots and save to the Plots folder

%load_ext autoreload
%autoreload 2

from JIF_plot_functions import JIF_graph_func

JIF_graph_func(JIF_sub_group_inf, DATA_TYPE_PREFIX)



### The below cells are not needed for generating the results

In [ ]:

# Investigate the result
con = sqlite3.connect(db_name)

#df = pd.read_sql_query("SELECT * FROM pub LIMIT 10;", con)
#df = pd.read_sql_query("SELECT * FROM jcr_score LIMIT 10;", con)
#print(df.info())

df = pd.read_sql_query("SELECT [match_method],count(*) FROM pub GROUP BY [match_method];", con)
print(df)
print()

df = pd.read_sql_query("SELECT [ISSN],[ISSN-L],[match_method],[match_val] FROM pub WHERE match_method='Journal:Abbrev' LIMIT 100;", con)
#print(df.head(50))

#df = pd.read_sql_query("SELECT * FROM jcr_score WHERE eISSN IS not NULL;", con)
#print(df.head())
